In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

from jupyterthemes import jtplot
jtplot.style()

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

from talib.abstract import *
import pandas as pd
import os
import numpy as np
from datetime import datetime
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('merged_2010_2016.csv')
df.head()

,Ticker,Date,Open,High,Low,Close,Volume
0,ENGY,2010-01-04,4.16,4.16,4.07,4.13,321300
1,TDEX,2010-01-04,5.22,5.22,5.14,5.18,2161600
2,2S,2010-01-04,3.10,3.14,3.10,3.10,379500
3,A,2010-01-04,3.66,3.66,3.60,3.60,66000
4,ACC,2010-01-04,2.14,2.14,2.14,2.14,100


In [3]:
df = pd.read_csv('merged_2010_2016.csv')
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')

df = df.loc[(df['Date'].dt.year==2016) & (df['Ticker']=='ADVANC')]
df = df.set_index('Date')
df.head()

,Ticker,Open,High,Low,Close,Volume
Date,,,,,,
2016-01-04,ADVANC,149.0,149.5,141.0,143.0,24880200
2016-01-05,ADVANC,144.0,144.5,131.0,131.0,59681500
2016-01-06,ADVANC,128.5,139.0,128.0,137.5,59203500
2016-01-07,ADVANC,133.0,138.0,131.5,134.5,32742400
2016-01-08,ADVANC,137.0,144.0,135.0,144.0,26585900


## Initial candlestick chart

In [4]:
INCREASING_COLOR = '#17BECF'
DECREASING_COLOR = '#7F7F7F'

In [5]:
data = [ dict(
    type = 'candlestick',
    open = df.Open,
    high = df.High,
    low = df.Low,
    close = df.Close,
    x = df.index,
    yaxis = 'y2',
    name = 'GS',
#     increasing = dict( line = dict( color = INCREASING_COLOR ) ),
#     decreasing = dict( line = dict( color = DECREASING_COLOR ) ),
) ]

layout=dict()

fig = dict(data=data, layout=layout)

## Create the layout object

In [18]:
fig['layout'] = dict()
# fig['layout']['paper_bgcolor'] = 'rgba(0,0,0,0)'
# fig['layout']['plot_bgcolor'] = 'rgba(0,0,0,0)'
fig['layout']['xaxis'] = dict( rangeselector = dict( visible = True ) )
fig['layout']['yaxis'] = dict( domain = [0, 0.2], showticklabels = False )
fig['layout']['yaxis2'] = dict( domain = [0.2, 0.8] )
fig['layout']['legend'] = dict( orientation = 'h', y=0.9, x=0.3, yanchor='bottom' )
fig['layout']['margin'] = dict( t=40, b=40, r=40, l=40 )

## Add range buttons

In [19]:
rangeselector=dict(
    visibe = True,
    x = 0, y = 0.9,
    bgcolor = 'rgba(150, 200, 250, 0.4)',
    font = dict( size = 13 ),
    buttons=list([
        dict(count=1,
             label='reset',
             step='all'),
        dict(count=1,
             label='1yr',
             step='year',
             stepmode='backward'),
        dict(count=3,
            label='3 mo',
            step='month',
            stepmode='backward'),
        dict(count=1,
            label='1 mo',
            step='month',
            stepmode='backward'),
        dict(step='all')
    ]))
    
fig['layout']['xaxis']['rangeselector'] = rangeselector

## Add moving average

In [20]:
sma15 = SMA(df, timeperiod=15, price='Close')

fig['data'].append( dict( x=sma15.index, y=sma15.values, type='scatter', mode='lines', 
                         line = dict( width = 1),
                         marker = dict( color = '#E377C2' ),
                         yaxis = 'y2', name='Moving Average' ) )

sma30 = SMA(df, timeperiod=30, price='Close')

fig['data'].append( dict( x=sma30.index, y=sma30.values, type='scatter', mode='lines', 
                         line = dict( width = 1),
                         marker = dict( color = '#E377C2' ),
                         yaxis = 'y2', name='Moving Average' ) )

## Set volume bar chart colors

In [21]:
colors = []

for i in range(len(df.Close)):
    if i != 0:
        if df.Volume[i] > df.Volume[i-1]:
            colors.append(INCREASING_COLOR)
        else:
            colors.append(DECREASING_COLOR)
    else:
        colors.append(DECREASING_COLOR)

## Add volume bar chart

In [22]:
fig['data'].append( dict( x=df.index, y=df.Volume,                         
                         marker=dict( color=colors ),
                         type='bar', yaxis='y', name='Volume' ) )

## Add bollinger bands


In [23]:
iplot( fig, filename = 'candlestick', validate = False )